# Double Pendulum Simulation
### by Ganden Schaffner and Ian McDougall

In [ ]:
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.integrate as integrate
from matplotlib.widgets import Slider

%matplotlib notebook

def derivs(state, t, G,th1,th2,L1,L2,M1,M2):
    dxdt = np.zeros_like(state) # initialize dervative array (values correspond to the time derivatives of initial state array)

    dxdt[0] = state[1] # derivative of angle1

    th_diff_init = state[2] - state[0] # difference of angle1 and angle2

    denom1 = (M1 + M2)*L1 - M2*L1*cos(th_diff_init)*cos(th_diff_init) # denominator of dxdt[2]
    dxdt[1] = (M2*L1*state[1]*state[1]*sin(th_diff_init)*cos(th_diff_init) +
               M2*G*sin(state[2])*cos(th_diff_init) +
               M2*L2*state[3]*state[3]*sin(th_diff_init) -
               (M1 + M2)*G*sin(state[0]))/denom1 # time derivative of angular velocity 1

    dxdt[2] = state[3] # time derivative of angle2

    denom2 = (L2/L1)*denom1 # denominator of dxdt[3]
    dxdt[3] = (-M2*L2*state[3]*state[3]*sin(th_diff_init)*cos(th_diff_init) +
               (M1 + M2)*G*sin(state[0])*cos(th_diff_init) -
               (M1 + M2)*L1*state[1]*state[1]*sin(th_diff_init) -
               (M1 + M2)*G*sin(state[2]))/denom2 # time derivative of angular velocity 2

    return dxdt # return the array of time derivatives

# Set default initial conditions
th1 = 175.0  # th1 and th2 are the initial angles (degrees)
th2 = 0.0
w1 = 0.0     # w10 and w20 are the initial angular velocities (degrees per second)
w2 = 0.0

G =  9.81     # acceleration due to gravity (in m/s^2)
L1 = 1.0      # length of pendulum (1 in m)
L2 = 1.0      # length of pendulum 2 (in m)
M1 = 1.0      # mass of pendulum 1 (in kg)
M2 = 10.0      # mass of pendulum 2 (in kg)
total_t = 20  # length of the animation

# Declare these variables so they may be used as globals
state,dt,y,x1,x2,y1,y2 = 0,0,0,0,0,0,0

def calculate(G,th1,th2,L1,L2,M1,M2):
    global state
    state = np.radians([th1, w1, th2, w2])
    
    global dt
    dt = 0.025 # Interval between successive pendulum states to calculate
    t = np.arange(0.0, total_t, dt)

    # Calculate the pendulum's position throughout the animation
    global y, x1, y1, x2, y2
    y = integrate.odeint(derivs, state, t, args=(G,th1,th2,L1,L2,M1,M2))

    x1 = L1*sin(y[:, 0])
    y1 = -L1*cos(y[:, 0])
    x2 = L2*sin(y[:, 2]) + x1
    y2 = -L2*cos(y[:, 2]) + y1

# Do the initial calculations
calculate(G,th1,th2,L1,L2,M1,M2)

# Set up the figure, axis, and line using the default initial conditions
fig = plt.figure(figsize=(8,8))
max_len = (L1 + L2)*1.1
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-max_len, max_len), ylim=(-max_len, max_len)) # 111 means 1x1 grid, first subplot
plt.subplots_adjust(left=0.1, bottom=0.3) # make space for the sliders
line, = ax.plot([], [], 'o-', lw=2) # o- sets a solid line and circular markers
time_template = 'time = %.1fs' # format as a float to the tenths place
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

'''
Sliders
'''
# The value must be taken (as this is called by slider.on_changed), but it is disregarded
def update(val):
    G = sld_G.val
    th1 = sld_th1.val
    th2 = sld_th2.val
    L1 = sld_L1.val
    L2 = sld_L2.val
    M1 = sld_M1.val
    M2 = sld_M2.val
    
    # Recompute
    calculate(G,th1,th2,L1,L2,M1,M2)
    
    # Adjust axes
    max_len = (L1 + L2)*1.1
    ax.axis([-max_len, max_len, -max_len, max_len])
    
    # Kill the old anim and make a new one
    global anim
    anim.event_source.stop()
    anim = animation.FuncAnimation(fig, animate, np.arange(1, len(y)), interval=25, blit=True, init_func=init_anim)
    
    plt.draw()

sld_pos_y, sld_i, sld_h = 0.02, 0, 0.02

# M2
sld_i += 1
ax_M2 = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_M2 = Slider(ax_M2, 'Pend. 2 Mass', 0, 20, valinit=M2, valfmt='%0.2f kg')
sld_M2.on_changed(update)
# M1
sld_i += 1
ax_M1 = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_M1 = Slider(ax_M1, 'Pend. 1 Mass', 0, 20, valinit=M1, valfmt='%0.2f kg')
sld_M1.on_changed(update)
# L2
sld_i += 1
ax_L2 = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_L2 = Slider(ax_L2, 'Pend. 2 Length', 0.1, 20, valinit=L2, valfmt='%0.2f m')
sld_L2.on_changed(update)
# L1
sld_i += 1
ax_L1 = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_L1 = Slider(ax_L1, 'Pend. 1 Length', 0.1, 20, valinit=L1, valfmt='%0.2f m')
sld_L1.on_changed(update)
# th2
sld_i += 1
ax_th2 = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_th2 = Slider(ax_th2, 'Pend. 2 Initial Angle', -180, 180, valinit=th2, valfmt='%0.2f º')
sld_th2.on_changed(update)
# th1
sld_i += 1
ax_th1 = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_th1 = Slider(ax_th1, 'Pend. 1 Initial Angle', -180, 180, valinit=th1, valfmt='%0.2f º')
sld_th1.on_changed(update)
# G
sld_i += 1
ax_G = plt.axes([0.2, sld_pos_y+sld_i*(0.01+sld_h), 0.65, sld_h])
sld_G = Slider(ax_G, 'Accel. Due to Gravity', 0.1, 30, valinit=G, valfmt='%0.2f N/kg')
sld_G.on_changed(update)

'''
Animation
'''
def set_plot_elements(x, y, text):
    global line,time_text
    line.set_data(x, y)
    time_text.set_text(text)
    return line, time_text

def init_anim():
    plt_elems = set_plot_elements([], [], '')
    return plt_elems

# Animates the line (this is called sequentially)
def animate(i):
    newx = [0, x1[i], x2[i]]
    newy = [0, y1[i], y2[i]]

    plt_elems = set_plot_elements(newx, newy, time_template % (i*dt))
    return plt_elems

# Call the animator. Only re-draw the parts that have changed (via blit=True)
anim = animation.FuncAnimation(fig, animate, np.arange(1, len(y)), interval=25, blit=True, init_func=init_anim)
plt.show()